In [3]:
import time
import threading
from collections import deque

import cv2
import numpy as np
import mediapipe as mp

In [4]:
# Optional beep on Windows; safe fallback if not available
try:
    import winsound
    def beep(freq=2000, dur_ms=300):
        winsound.Beep(freq, dur_ms)
except Exception:
    def beep(freq=2000, dur_ms=300):
        pass

# ===================== Settings =====================
CAMERA_INDEX = 0
FRAME_RESIZE = 1.0         # 1.0 = full res; reduce to 0.75 / 0.5 if slow
EAR_THRESHOLD = 0.25       # lower = stricter; typical 0.25–0.30 for MediaPipe
DROWSY_FRAMES = 20         # consecutive frames below threshold to trigger
SMOOTH_WINDOW = 5          # moving average for EAR smoothing
ALERT_COOLDOWN_S = 1.5     # min seconds between beeps
DRAW_DEBUG = True
# =====================================================

# MediaPipe face mesh with iris helps stable eye landmarks
mp_face = mp.solutions.face_mesh

# Eye landmark indices (MediaPipe FaceMesh)
# Using 6 points per eye to compute EAR: (p1,p2,p3,p4,p5,p6)
# Mapping chosen for stability (outer to inner corners, upper/lower lids).
LEFT_EYE = [33, 160, 158, 133, 153, 144]   # p1=33, p4=133 corners
RIGHT_EYE = [263, 387, 385, 362, 380, 373] # p1=263, p4=362 corners

def euclidean_dist(a, b):
    return np.linalg.norm(a - b)

def compute_ear(landmarks_xy, eye_indices):
    # landmarks_xy: (N, 2) in pixel coords; eye_indices: 6 indices
    p = [landmarks_xy[i] for i in eye_indices]
    p1, p2, p3, p4, p5, p6 = p
    # EAR: (||p2 - p6|| + ||p3 - p5||) / (2 * ||p1 - p4||)
    num = euclidean_dist(p2, p6) + euclidean_dist(p3, p5)
    den = 2.0 * euclidean_dist(p1, p4)
    if den <= 1e-6:
        return 0.0
    return float(num / den)

def draw_eye_polygon(frame, landmarks_xy, idxs, color):
    pts = np.array([landmarks_xy[i] for i in idxs], dtype=np.int32)
    cv2.polylines(frame, [pts], isClosed=True, color=color, thickness=1)

def throttled_beep(last_time_holder, cooldown_s):
    t = time.time()
    if t - last_time_holder[0] >= cooldown_s:
        last_time_holder[0] = t
        threading.Thread(target=beep, kwargs={"freq": 2000, "dur_ms": 250}, daemon=True).start()

cap = cv2.VideoCapture(CAMERA_INDEX)
if not cap.isOpened():
    raise RuntimeError("Could not open webcam. Check index/permissions.")

# Prepare face mesh
face_mesh = mp_face.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,   # iris landmarks for better eyes
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

ear_hist = deque(maxlen=SMOOTH_WINDOW)
frames_below = 0
last_beep_at = [0.0]
font = cv2.FONT_HERSHEY_SIMPLEX

try:
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        if FRAME_RESIZE != 1.0:
            frame = cv2.resize(frame, (0, 0), fx=FRAME_RESIZE, fy=FRAME_RESIZE)

        h, w = frame.shape[:2]
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = face_mesh.process(rgb)

        ear = None
        drowsy = False

        if res.multi_face_landmarks:
            lm = res.multi_face_landmarks[0].landmark
            # Convert to pixel coords
            landmarks_xy = np.array([[lm_i.x * w, lm_i.y * h] for lm_i in lm], dtype=np.float32)

            left_ear = compute_ear(landmarks_xy, LEFT_EYE)
            right_ear = compute_ear(landmarks_xy, RIGHT_EYE)
            ear = (left_ear + right_ear) / 2.0

            ear_hist.append(ear)
            ear_smooth = float(np.mean(ear_hist))
            below = ear_smooth < EAR_THRESHOLD

            if DRAW_DEBUG:
                # Draw eyes and EAR
                draw_eye_polygon(frame, landmarks_xy, LEFT_EYE, (0, 255, 255))
                draw_eye_polygon(frame, landmarks_xy, RIGHT_EYE, (0, 255, 255))
                cv2.putText(frame, f"EAR: {ear_smooth:.3f}", (10, 30), font, 0.8, (0, 255, 0) if not below else (0, 0, 255), 2)

            if below:
                frames_below += 1
            else:
                frames_below = 0

            if frames_below >= DROWSY_FRAMES:
                drowsy = True

        else:
            # No face -> reset counters, show hint
            frames_below = 0
            ear_hist.clear()
            cv2.putText(frame, "No face detected", (10, 30), font, 0.8, (0, 255, 255), 2)

        # Alerts
        if drowsy:
            cv2.rectangle(frame, (0, 0), (w, 60), (0, 0, 255), -1)
            cv2.putText(frame, "DROWSINESS ALERT!", (10, 40), font, 1.0, (255, 255, 255), 3)
            throttled_beep(last_beep_at, ALERT_COOLDOWN_S)
        else:
            cv2.putText(frame, "Status: OK", (10, 60), font, 0.7, (0, 200, 0), 2)

        # UI
        cv2.putText(frame, "Press Q to quit", (10, h - 12), font, 0.6, (200, 200, 200), 1)
        cv2.imshow("Driver Drowsiness Detection (EAR)", frame)
        if cv2.waitKey(1) & 0xFF in (ord('q'), ord('Q')):
            break

finally:
    face_mesh.close()
    cap.release()
    cv2.destroyAllWindows()